In [1]:
# Imports
import cv2
import numpy as np
import json
import time
from tensorflow.keras.models import load_model

In [2]:
# Configuration
MODEL_PATH = 'face_model.keras'
CLASS_NAMES_PATH = 'class_names_student.json'
IMG_SIZE = 128

# Load Model
try:
    model = load_model(MODEL_PATH)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")

# Load Class Names
try:
    with open(CLASS_NAMES_PATH, 'r') as f:
        class_names = json.load(f)
    print(f"Loaded {len(class_names)} classes.")
except Exception as e:
    print(f"Error loading class names: {e}")

# Load Face Detector (Haar Cascade as used in training)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

Model loaded successfully.
Loaded 35 classes.


In [ ]:
# Webcam Capture
cap = cv2.VideoCapture(0)

last_time = time.time()
label_text = "?"
confidence = 0.0

print("Starting Webcam... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 1. Convert to Grayscale for Detection (same as Assignment_3)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 2. Detect Faces
    faces = face_cascade.detectMultiScale(
        gray, 
        scaleFactor=1.1, 
        minNeighbors=4, 
        minSize=(30, 30)
    )

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract Region of Interest (ROI)
        face_roi = frame[y:y+h, x:x+w]
        
        # 3. Preprocessing (Resize, Normalize, Expand Dims)
        try:
            # Resize to 128x128
            resized = cv2.resize(face_roi, (IMG_SIZE, IMG_SIZE))
            
            # Convert BGR to RGB (Model expects RGB)
            rgb_face = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
            
            # Normalize to [0, 1]
            normalized = rgb_face.astype("float32") / 255.0
            
            # Expand dimensions to create batch (1, 128, 128, 3)
            batch_input = np.expand_dims(normalized, axis=0)

            # 4. Prediction
            # Update prediction every few frames to reduce lag if needed, 
            # or predict every frame if performance allows.
            preds = model.predict(batch_input, verbose=0)
            idx = np.argmax(preds)
            confidence = np.max(preds) * 100
            
            if idx < len(class_names):
                label_text = class_names[idx]
            else:
                label_text = "Unknown"
                
            # Console Output
            print(f"Detected: {label_text} ({confidence:.2f}%)")
            
            # Draw Bounding Box and Label
            color = (0, 255, 0) # Green
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, f"{label_text} {confidence:.1f}%", (x, y-10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                        
        except Exception as e:
            print(f"Error processing face: {e}")

    # Display Result
    cv2.imshow("Face Recognition Demo", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Starting Webcam... Press 'q' to quit.
Detected: 22-49037-3 (49.42%)
Detected: 22-49783-3 (60.56%)
Detected: 22-49037-3 (62.27%)
Detected: 22-48582-3 (45.19%)
Detected: 22-49037-3 (62.22%)
Detected: 22-48582-3 (42.70%)
Detected: 22-49783-3 (49.04%)
Detected: 22-49037-3 (77.95%)
Detected: 22-49453-3 (21.23%)
Detected: 22-48582-3 (51.63%)
Detected: 22-49037-3 (26.90%)
Detected: 22-49037-3 (31.35%)
Detected: 22-48582-3 (37.97%)
Detected: 22-49453-3 (58.30%)
Detected: 22-49453-3 (31.83%)
Detected: 22-47813-2 (30.03%)
Detected: 22-48582-3 (20.64%)
Detected: 22-48582-3 (51.64%)
Detected: 22-48582-3 (73.86%)
Detected: 22-48582-3 (57.98%)
Detected: 22-48582-3 (86.04%)
Detected: 22-48005-2 (28.10%)
Detected: 22-49453-3 (21.51%)
Detected: 22-49453-3 (26.50%)
Detected: 22-49453-3 (19.03%)
Detected: 22-48005-2 (16.42%)
Detected: 22-49037-3 (21.32%)
Detected: 22-48005-2 (20.99%)
Detected: 22-49453-3 (41.19%)
Detected: 22-49453-3 (14.41%)
Detected: 22-48005-2 (18.27%)
Detected: 22-48582-3 (58.47%)
De